In [1]:
using PhyloClustering, PhyloNetworks, Clustering, CSV, DataFrames, JLD, SparseArrays, Suppressor, Distances, Hungarian, MLBase, LinearAlgebra

In [375]:
tree_100_1 = jldopen("../../data/trees/8_diff_topo_12_100_1.jld", "r") do file
    read(file,"tree")
end
tree_100_2 = jldopen("../../data/trees/8_diff_topo_13_100_1.jld", "r") do file
    read(file,"tree")
end
tree_100_1  = DataFrame(tree_100_1,[string(i) for i in 0:126])
tree_100_2  = DataFrame(tree_100_2,[string(i) for i in 0:126])
tree_1000_1 = CSV.read("../../data/trees/8_diff_topo_12_1000_1.csv", DataFrame);
tree_1000_2 = CSV.read("../../data/trees/8_diff_topo_13_1000_1.csv", DataFrame);
tree_5000_1 = CSV.read("../../data/trees/8_diff_topo_12_5000_1.csv", DataFrame);
tree_5000_2 = CSV.read("../../data/trees/8_diff_topo_13_5000_1.csv", DataFrame);

In [376]:
tree_100 = vcat(tree_100_1,tree_100_2);
tree_1000 = vcat(tree_1000_1,tree_1000_2);
tree_5000 = vcat(tree_5000_1,tree_5000_2);

In [377]:
tree_100 = standardize_tree(Matrix(tree_100));
tree_1000 = standardize_tree(Matrix(tree_1000));
tree_5000 = standardize_tree(Matrix(tree_5000));

In [378]:
distance_100 = distance(tree_100);
distance_1000 = distance(tree_1000);
distance_5000 = distance(tree_5000);

In [379]:
@suppress_err begin
    @time kmeans_label(tree_100, 2);
    @time hc_label(distance_100, 2);
    @time gmm_label(tree_100, 2);
end

  0.000909 seconds (31 allocations: 14.875 KiB)
  0.000831 seconds (485 allocations: 771.031 KiB)
K-means converged with 2 iterations (objv = 7543.253441070356)
  0.013281 seconds (8.87 k allocations: 5.714 MiB)


([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [380]:
@suppress_err begin
    @time  kmeans_label(tree_1000, 2);
    @time hc_label(distance_1000, 2);
    @time  gmm_label(tree_1000, 2);
end

  0.000546 seconds (31 allocations: 70.812 KiB)
  0.071734 seconds (4.80 k allocations: 62.603 MiB)
K-means converged with 3 iterations (objv = 123575.60915543442)
  0.041671 seconds (12.37 k allocations: 52.877 MiB)


([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [381]:
@suppress_err begin
    @time  kmeans_label(tree_5000, 2);
    @time hc_label(distance_5000, 2);
    @time  gmm_label(tree_5000, 2);
end

  0.005612 seconds (35 allocations: 320.500 KiB)
  4.164227 seconds (24.27 k allocations: 1.499 GiB, 0.58% gc time)
K-means converged with 4 iterations (objv = 166274.58455114532)
  0.437348 seconds (28.42 k allocations: 237.704 MiB, 62.78% gc time)


([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [480]:
topo_4_100 = readMultiTopology("../../data/rawTree/4_diff_topo_1_100_1.trees");
topo_4_5000 = readMultiTopology("../../data/rawTree/4_diff_topo_1_5000_1.trees");
topo_8_100 = readMultiTopology("../../data/rawTree/8_diff_topo_1_100_1.trees");
topo_8_5000 = readMultiTopology("../../data/rawTree/8_diff_topo_1_5000_1.trees");
topo_16_100 = readMultiTopology("../../data/rawTree/16_diff_topo_1_100_1.trees");
topo_16_5000 = readMultiTopology("../../data/rawTree/16_diff_topo_1_5000_1.trees");

In [482]:
@time split_weight(topo_4_100, 4);
@time split_weight(topo_4_5000, 4);
@time split_weight(topo_8_100, 8);
@time split_weight(topo_8_5000, 8);
@time split_weight(topo_16_100, 16);
@time split_weight(topo_16_5000, 16);

  0.001554 seconds (18.26 k allocations: 904.969 KiB)
  0.059148 seconds (968.13 k allocations: 45.069 MiB, 12.52% gc time)
  0.016800 seconds (91.72 k allocations: 4.971 MiB, 70.09% gc time)
  0.314500 seconds (4.74 M allocations: 252.612 MiB, 18.56% gc time)
  0.330106 seconds (6.45 M allocations: 474.276 MiB, 24.16% gc time)
 17.310758 seconds (321.89 M allocations: 23.181 GiB, 23.02% gc time)


### Compare

In [4]:
function ground_true(df_1, df_2)
    a = fill(1,size(df_1)[1])
    b = fill(2,size(df_2)[1])
    gt = cat(a,b, dims = 1)
    return gt
end

function accuracy(n, gt, pred)
    matrix = confusmat(n, gt, pred)
    # Hungarian algorithm minimizes the cost, so we need to transform the matrix
    A = -matrix .+ maximum(matrix)    
    matrix = matrix[:,hungarian(A)[1]]
    x = tr(matrix)/sum(matrix)
    return matrix, x
end

accuracy (generic function with 1 method)

In [483]:
trees_1000_3 = readMultiTopology("../../data/rawTree/4_diff_topo_1_1000_1.trees");
trees_1000_4 = readMultiTopology("../../data/rawTree/4_diff_topo_5_1000_1.trees");

In [484]:
@time begin
    trees_3 = split_weight(trees_1000_3, 4);
    trees_4 = split_weight(trees_1000_4, 4);
    trees = vcat(trees_3, trees_4)
    distance_matrix = zeros(2000, 2000)
    for i in 1:2000
        for j in i:2000
            distance_matrix[i,j] = euclidean(trees[i,:], trees[j,:])
            distance_matrix[j,i] = distance_matrix[i,j]
        end
    end
end

gt = ground_true(trees_1000_3, trees_1000_4);
label = hc_label(distance_matrix, 2);
accuracy(2, gt, label)

  1.906814 seconds (30.26 M allocations: 870.880 MiB, 7.22% gc time)


([957 43; 73 927], 0.942)

In [486]:
# Pairwise distance between trees in trees_2000
@time begin
    trees_2000 = reduce(vcat, (trees_1000_3, trees_1000_4));
    matrix = zeros(2000, 2000)
    for i in 1:2000
        for j in i:2000
            matrix[i,j] = hardwiredClusterDistance(trees_2000[i], trees_2000[j], true)
            matrix[j,i] = matrix[i,j]
        end
    end
end

label = hc_label(matrix, 2);
randindex(gt, label)[1]
accuracy(2, gt, label)

 11.192609 seconds (87.49 M allocations: 5.295 GiB, 7.11% gc time)


([914 86; 60 940], 0.927)

In [487]:
trees_1000_3 = readMultiTopology("../../data/rawTree/8_diff_topo_1_1000_1.trees");
trees_1000_4 = readMultiTopology("../../data/rawTree/8_diff_topo_5_1000_1.trees");

In [488]:
@time begin
    trees_3 = split_weight(trees_1000_3, 8);
    trees_4 = split_weight(trees_1000_4, 8);
    trees = vcat(trees_3, trees_4)
    distance_matrix = zeros(2000, 2000)
    for i in 1:2000
        for j in i:2000
            distance_matrix[i,j] = euclidean(trees[i,:], trees[j,:])
            distance_matrix[j,i] = distance_matrix[i,j]
        end
    end
end

gt = ground_true(trees_1000_3, trees_1000_4);
label = hc_label(distance_matrix, 2);
accuracy(2, gt, label)

  5.929103 seconds (31.77 M allocations: 4.572 GiB, 11.91% gc time)


([998 2; 6 994], 0.996)

In [490]:
# Pairwise distance between trees in trees_2000
@time begin
    trees_2000 = reduce(vcat, (trees_1000_3, trees_1000_4));
    matrix = zeros(2000, 2000)
    for i in 1:2000
        for j in i:2000
            matrix[i,j] = hardwiredClusterDistance(trees_2000[i], trees_2000[j], true)
            matrix[j,i] = matrix[i,j]
        end
    end
end

label = hc_label(matrix, 2);
accuracy(2, gt, label)

 37.146473 seconds (259.03 M allocations: 25.557 GiB, 8.76% gc time)


([1000 0; 1 999], 0.9995)

### Imbalanced Data

In [348]:
trees_1 = vcat(tree_100_1, tree_1000_2);
trees_1 = standardize_tree(Matrix(trees_1));
gt_1 = ground_true(tree_100_1, tree_1000_2);
trees_2 = vcat(tree_1000_1, tree_5000_2);
trees_2 = standardize_tree(Matrix(trees_2));
gt_2 = ground_true(tree_1000_1, tree_5000_2);

In [349]:
@suppress_err begin
    high_k = 0
    high_g = 0
    for i in 1:10
        label_k = kmeans_label(trees_1, 2);
        label_g, _ = gmm_label(trees_1, 2);
        k = randindex(gt_1, label_k)[1]
        g = randindex(gt_1, label_g)[1]
        if k > high_k
            high_k = k
        end  
        if g > high_g
            high_g = g
        end
    end
    label_h = hc_label(distance(trees_1), 2);
    high_h = randindex(gt_1, label_h)[1]
    print("Kmeans: ", high_k, "\n")
    print("GMM: ", high_g, "\n")
    print("HC: ", high_h, "\n")
end


K-means converged with 2 iterations (objv = 53744.243875040265)
K-means converged with 2 iterations (objv = 53854.066190899684)
K-means converged with 2 iterations (objv = 53831.31953511836)
K-means converged with 2 iterations (objv = 53967.24674868098)


K-means converged with 5 iterations (objv = 52220.1657578649)
K-means converged with 12 iterations (objv = 52479.9214701662)
K-means converged with 16 iterations (objv = 52480.056789688664)
K-means converged with 2 iterations (objv = 53394.12508940341)
K-means converged with 14 iterations (objv = 53257.235117451855)
K-means converged with 2 iterations (objv = 54043.57337562963)
Kmeans: 0.9668381635085826
GMM: 0
HC: 0.9082465689958767


In [350]:
@suppress_err begin
    high_k = 0
    high_g = 0
    for i in 1:10
        label_k = kmeans_label(trees_2, 2);
        label_g, _ = gmm_label(trees_2, 2);
        k = randindex(gt_2, label_k)[1]
        g = randindex(gt_2, label_g)[1]
        if k > high_k
            high_k = k
        end  
        if g > high_g
            high_g = g
        end
    end
    label_h = hc_label(distance(trees_2), 2);
    high_h = randindex(gt_2, label_h)[1]
    print("Kmeans: ", high_k, "\n")
    print("GMM: ", high_g, "\n")
    print("HC: ", high_h, "\n")
end

K-means converged with 2 iterations (objv = 119285.49975679301)
K-means converged with 31 iterations (objv = 139588.45141041756)
K-means converged with 13 iterations (objv = 146031.86721906636)
K-means converged with 4 iterations (objv = 158394.91811491392)
K-means converged with 2 iterations (objv = 132774.15774579748)
K-means converged with 2 iterations (objv = 144819.708595497)
K-means converged with 2 iterations (objv = 121266.73584726526)
K-means converged with 35 iterations (objv = 138437.8376340382)
K-means converged with 14 iterations (objv = 133927.67256806165)
K-means converged with 2 iterations (objv = 133454.65845988446)
Kmeans: 0.952254595835525
GMM: 0
HC: 0.9209969830160794


In [2]:
tree_100_1 = CSV.read("../../data/trees/4_diff_topo_1_100_1.csv", DataFrame);
tree_100_2 = CSV.read("../../data/trees/4_diff_topo_5_100_1.csv", DataFrame);
tree_1000_1 = CSV.read("../../data/trees/4_diff_topo_1_1000_1.csv", DataFrame);
tree_1000_2 = CSV.read("../../data/trees/4_diff_topo_5_1000_1.csv", DataFrame);
tree_5000_1 = CSV.read("../../data/trees/4_diff_topo_1_5000_1.csv", DataFrame);
tree_5000_2 = CSV.read("../../data/trees/4_diff_topo_5_5000_1.csv", DataFrame);

In [5]:
trees_1 = vcat(tree_100_1, tree_1000_2);
trees_1 = standardize_tree(Matrix(trees_1));
gt_1 = ground_true(tree_100_1, tree_1000_2);
trees_2 = vcat(tree_1000_1, tree_5000_2);
trees_2 = standardize_tree(Matrix(trees_2));
gt_2 = ground_true(tree_1000_1, tree_5000_2);

In [6]:
@suppress_err begin
    high_k = 0
    high_g = 0
    for i in 1:10
        label_k = kmeans_label(trees_1, 2);
        label_g, _ = gmm_label(trees_1, 2);
        k = randindex(gt_1, label_k)[1]
        g = randindex(gt_1, label_g)[1]
        if k > high_k
            high_k = k
        end  
        if g > high_g
            high_g = g
        end
    end
    label_h = hc_label(distance(trees_1), 2);
    high_h = randindex(gt_1, label_h)[1]
    print("Kmeans: ", high_k, "\n")
    print("GMM: ", high_g, "\n")
    print("HC: ", high_h, "\n")
end


K-means converged with 24 iterations (objv = 5752.844794826464)
K-means converged with 10 iterations (objv = 5752.845767301914)
K-means converged with 8 iterations (objv = 5752.845767301914)
K-means converged with 19 iterations (objv = 5752.844794826464)
K-means converged with 7 iterations (objv = 5752.844794826464)
K-means converged with 22 iterations (objv = 5752.844794826464)
K-means converged with 20 iterations (objv = 5752.844794826464)
K-means converged with 4 iterations (objv = 6564.915570090179)
K-means converged with 8 iterations (objv = 5752.845767301914)
K-means converged with 22 iterations (objv = 5752.844794826464)
Kmeans: 0.32581356865411054
GMM: 0.5703676242832509
HC: 0.3761479399059546


In [7]:
@suppress_err begin
    high_k = 0
    high_g = 0
    for i in 1:10
        label_k = kmeans_label(trees_2, 2);
        label_g, _ = gmm_label(trees_2, 2);
        k = randindex(gt_2, label_k)[1]
        g = randindex(gt_2, label_g)[1]
        if k > high_k
            high_k = k
        end  
        if g > high_g
            high_g = g
        end
    end
    label_h = hc_label(distance(trees_2), 2);
    high_h = randindex(gt_2, label_h)[1]
    print("Kmeans: ", high_k, "\n")
    print("GMM: ", high_g, "\n")
    print("HC: ", high_h, "\n")
end

K-means converged with 13 iterations (objv = 10230.301252330677)
K-means converged with 20 iterations (objv = 11372.279698075545)
K-means converged with 23 iterations (objv = 9531.982053410375)
K-means converged with 6 iterations (objv = 12187.245960275122)
K-means converged with 16 iterations (objv = 10902.616385361476)
K-means converged with 11 iterations (objv = 11072.970862952816)
K-means converged with 17 iterations (objv = 10223.499961788828)
K-means converged with 11 iterations (objv = 10257.140043230158)
K-means converged with 8 iterations (objv = 13483.416207308499)
K-means converged with 7 iterations (objv = 12190.136777073241)
Kmeans: 0.4061613897317699
GMM: 0.6720767445910563
HC: 0.3986519096443959


### Rand index

In [263]:
function rep_kmeans_matrix(trees, path)
    n = length(trees)
    result = zeros(n, n)
    for i in 2:n
        for j in  1:(i - 1)
            gt = ground_true(trees[i],trees[j])
            tree = vcat(trees[i],trees[j])
            tree = standardize_tree(Matrix(tree))
            for k in 1:10
                pred_kmeans = kmeans_label(tree, 2) 
                x = randindex(gt, pred_kmeans)[1]
                if x > result[i,j]
                    result[i,j] = x
                    result[j,i] = x
                end
            end
        end
    end      
    header = Vector(1:n)
    header = string.(header)
    CSV.write(path, DataFrame(result, :auto),header = header);
end

rep_kmeans_matrix (generic function with 1 method)

In [269]:
for i in 1:100
    trees = []
    for j in 1:15      
        path = "../../data/trees/8_diff_topo_" * string(j) *  "_1000_" * string(i) * ".csv"
        tree = CSV.read(path, DataFrame);
        push!(trees, tree)
    end
    rep_kmeans_matrix(trees, "../../data/result/randindex/kmeans_8_diff_topo_1000_" *string(i)* ".csv");
end